In [1]:
import collections

In [2]:
PAGE_SIZE_4K = 4 * 1024  
TLB_ENTRIES = 16         
INT_SIZE = 4             

In [3]:
class TLBSimulator:
    def __init__(self, tlb_size: int, page_size: int):            
        self.tlb_size = tlb_size
        self.page_size = page_size
        
        self.tlb = collections.OrderedDict()
        
        self.seen_pages = set() # conjunto para rastrear páginas já vistas
        
        self.total_accesses = 0
        self.tlb_hits = 0
        self.tlb_misses_compulsory = 0
        self.tlb_misses_capacity = 0

    def access_address(self, virtual_address: int):
        self.total_accesses += 1

        page_number = virtual_address // self.page_size
    
        if page_number in self.tlb:
            self.tlb_hits += 1
            self.tlb.move_to_end(page_number)
            return
        elif page_number not in self.seen_pages: # se a página nunca foi vista antes é um MISS COMPULSÓRIO
            self.tlb_misses_compulsory += 1
            self.seen_pages.add(page_number) # adiciona a página ao conjunto de páginas vistas
        else:
            self.tlb_misses_capacity += 1
            
        if len(self.tlb) >= self.tlb_size:
            self.tlb.popitem(last=False)
        self.tlb[page_number] = page_number
        
    def print_stats(self):
        total_misses = self.tlb_misses_compulsory + self.tlb_misses_capacity
        
        hit_rate = (self.tlb_hits / self.total_accesses * 100) if self.total_accesses > 0 else 0
        miss_rate = (total_misses / self.total_accesses * 100) if self.total_accesses > 0 else 0
        
        if total_misses > 0:
            ratio_non_compulsory = self.tlb_misses_capacity / total_misses * 100
        else:
            ratio_non_compulsory = 0
            
        print("-" * 40)
        print("Estatísticas da Simulação da TLB")
        print("-" * 40)
        print(f"Total de Acessos à Memória: {self.total_accesses:,}")
        print(f"Acertos (Hits) na TLB:       {self.tlb_hits:,} ({hit_rate:.2f}%)")
        print(f"Faltas (Misses) na TLB:      {total_misses:,} ({miss_rate:.2f}%)")
        print(f"  - Misses Compulsórios:     {self.tlb_misses_compulsory:,}")
        print(f"  - Misses de Capacidade:    {self.tlb_misses_capacity:,}")
        print("-" * 40)
        print(f"Razão (Não-Compulsórios / Total de Misses): {ratio_non_compulsory:.2f}%")
        print("-" * 40)

In [4]:
def run_experiment(matrix_dim, tlb_size, page_size, data_type_size=4):
    print(f"\nIniciando experimento: Matriz {matrix_dim}x{matrix_dim}, TLB com {tlb_size} entradas, Página de {page_size} bytes")
    
    simulator = TLBSimulator(tlb_size=tlb_size, page_size=page_size)
    for j in range(matrix_dim):
        for i in range(matrix_dim):
            index = i * matrix_dim + j
            virtual_address = index * data_type_size
            simulator.access_address(virtual_address)
            
    simulator.print_stats()

In [ ]:
def run_experiment_blocked(matrix_dim, tlb_size, page_size, block_size, data_type_size=4):
    print(f"Iniciando experimento: Matriz {matrix_dim}x{matrix_dim}, TLB com {tlb_size} entradas, Página de {page_size} bytes, Blocos de {block_size}x{block_size}")
    
    simulator = TLBSimulator(tlb_size=tlb_size, page_size=page_size)
    outs = []
    for i0 in range(0, matrix_dim, block_size): 
        for j0 in range(0, matrix_dim, block_size):
            for i in range(i0, min(i0 + block_size, matrix_dim)):  
                for j in range(j0, min(j0 + block_size, matrix_dim)): 
                    index = i * matrix_dim + j # índice linear
                    virtual_address = index * data_type_size
                    pages_number = virtual_address // page_size
                    simulator.access_address(virtual_address)
                    
    simulator.print_stats()
    return outs

In [6]:
for i0 in range(0, 4, 2):  # primeiro blocos de linha
        for j0 in range(0, 4, 2):  # depois blocos de coluna
            print(f"Bloco iniciando em [{i0},{j0}]")
            for i in range(i0, min(i0 + 2, 4)):  # percorre linha dentro do bloco
                for j in range(j0, min(j0 + 2, 4)):  # percorre coluna dentro do bloco
                    index = i * 4 + j
                    virtual_address = index * 4
                    pages_number = virtual_address // (4 * 1024)  
                    print(f"Acessando elemento [{i},{j}] no índice linear {index} - Endereço Virtual: {virtual_address} - Página: {pages_number}")

Bloco iniciando em [0,0]
Acessando elemento [0,0] no índice linear 0 - Endereço Virtual: 0 - Página: 0
Acessando elemento [0,1] no índice linear 1 - Endereço Virtual: 4 - Página: 0
Acessando elemento [1,0] no índice linear 4 - Endereço Virtual: 16 - Página: 0
Acessando elemento [1,1] no índice linear 5 - Endereço Virtual: 20 - Página: 0
Bloco iniciando em [0,2]
Acessando elemento [0,2] no índice linear 2 - Endereço Virtual: 8 - Página: 0
Acessando elemento [0,3] no índice linear 3 - Endereço Virtual: 12 - Página: 0
Acessando elemento [1,2] no índice linear 6 - Endereço Virtual: 24 - Página: 0
Acessando elemento [1,3] no índice linear 7 - Endereço Virtual: 28 - Página: 0
Bloco iniciando em [2,0]
Acessando elemento [2,0] no índice linear 8 - Endereço Virtual: 32 - Página: 0
Acessando elemento [2,1] no índice linear 9 - Endereço Virtual: 36 - Página: 0
Acessando elemento [3,0] no índice linear 12 - Endereço Virtual: 48 - Página: 0
Acessando elemento [3,1] no índice linear 13 - Endereço Vi

In [7]:
pn = run_experiment_blocked(matrix_dim=8*1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=1, data_type_size=INT_SIZE)
with open("pn5.txt", "w") as f:
    for pn in pn:
        f.write(f"{pn}\n")

Iniciando experimento: Matriz 8192x8192, TLB com 16 entradas, Página de 4096 bytes, Blocos de 1x1


KeyboardInterrupt: 

In [ ]:
run_experiment_blocked(matrix_dim=1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=4, data_type_size=INT_SIZE)

Iniciando experimento: Matriz 1024x1024, TLB com 16 entradas, Página de 4096 bytes, Blocos de 4x4
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 1,048,576
Acertos (Hits) na TLB:       1,047,552 (99.90%)
Faltas (Misses) na TLB:      1,024 (0.10%)
  - Misses Compulsórios:     1,024
  - Misses de Capacidade:    0
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 0.00%
----------------------------------------


['Bloco iniciando em [0,0]',
 'Acessando elemento [0,0] no índice linear 0 - Endereço Virtual: 0 - Página: 0',
 'Acessando elemento [0,1] no índice linear 1 - Endereço Virtual: 4 - Página: 0',
 'Acessando elemento [0,2] no índice linear 2 - Endereço Virtual: 8 - Página: 0',
 'Acessando elemento [0,3] no índice linear 3 - Endereço Virtual: 12 - Página: 0',
 'Acessando elemento [1,0] no índice linear 1024 - Endereço Virtual: 4096 - Página: 1',
 'Acessando elemento [1,1] no índice linear 1025 - Endereço Virtual: 4100 - Página: 1',
 'Acessando elemento [1,2] no índice linear 1026 - Endereço Virtual: 4104 - Página: 1',
 'Acessando elemento [1,3] no índice linear 1027 - Endereço Virtual: 4108 - Página: 1',
 'Acessando elemento [2,0] no índice linear 2048 - Endereço Virtual: 8192 - Página: 2',
 'Acessando elemento [2,1] no índice linear 2049 - Endereço Virtual: 8196 - Página: 2',
 'Acessando elemento [2,2] no índice linear 2050 - Endereço Virtual: 8200 - Página: 2',
 'Acessando elemento [2,3]

In [ ]:
run_experiment_blocked(matrix_dim=8*1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=4, data_type_size=INT_SIZE)

Iniciando experimento: Matriz 8192x8192, TLB com 16 entradas, Página de 4096 bytes, Blocos de 4x4
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 67,108,864
Acertos (Hits) na TLB:       50,331,648 (75.00%)
Faltas (Misses) na TLB:      16,777,216 (25.00%)
  - Misses Compulsórios:     65,536
  - Misses de Capacidade:    16,711,680
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 99.61%
----------------------------------------


In [ ]:
# vas_set = set(vas)
# if len(vas) != len(vas_set):
#     print("Há endereços virtuais reaproveitados (repetidos)!")
#     # Para ver quais são:
#     from collections import Counter
#     counts = Counter(vas)
#     repetidos = [addr for addr, count in counts.items() if count > 1]
#     print(f"Total de endereços repetidos: {len(repetidos)}")
#     print("Exemplos de endereços repetidos:", repetidos[:10])
# else:
#     print("Não há endereços virtuais reaproveitados (todos são únicos).")

In [ ]:
run_experiment_blocked(matrix_dim=1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=1, data_type_size=INT_SIZE)

Iniciando experimento: Matriz 1024x1024, TLB com 16 entradas, Página de 4096 bytes, Blocos de 1x1
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 1,048,576
Acertos (Hits) na TLB:       0 (0.00%)
Faltas (Misses) na TLB:      1,048,576 (100.00%)
  - Misses Compulsórios:     1,024
  - Misses de Capacidade:    1,047,552
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 99.90%
----------------------------------------


In [ ]:
run_experiment_blocked(matrix_dim=1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=2, data_type_size=INT_SIZE)

Iniciando experimento: Matriz 1024x1024, TLB com 16 entradas, Página de 4096 bytes, Blocos de 2x2
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 1,048,576
Acertos (Hits) na TLB:       524,288 (50.00%)
Faltas (Misses) na TLB:      524,288 (50.00%)
  - Misses Compulsórios:     1,024
  - Misses de Capacidade:    523,264
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 99.80%
----------------------------------------


In [ ]:
run_experiment_blocked(matrix_dim=1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, block_size=3, data_type_size=INT_SIZE)

Iniciando experimento: Matriz 1024x1024, TLB com 16 entradas, Página de 4096 bytes, Blocos de 3x3
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 1,048,576
Acertos (Hits) na TLB:       698,368 (66.60%)
Faltas (Misses) na TLB:      350,208 (33.40%)
  - Misses Compulsórios:     1,024
  - Misses de Capacidade:    349,184
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 99.71%
----------------------------------------


In [ ]:
# apenas misses compulsórios -- matriz pequena
run_experiment(matrix_dim=128, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, data_type_size=INT_SIZE)

# muitos misses de capacidade
run_experiment(matrix_dim=1024, tlb_size=TLB_ENTRIES, page_size=PAGE_SIZE_4K, data_type_size=INT_SIZE)


Iniciando experimento: Matriz 128x128, TLB com 16 entradas, Página de 4096 bytes
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 16,384
Acertos (Hits) na TLB:       16,368 (99.90%)
Faltas (Misses) na TLB:      16 (0.10%)
  - Misses Compulsórios:     16
  - Misses de Capacidade:    0
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 0.00%
----------------------------------------

Iniciando experimento: Matriz 1024x1024, TLB com 16 entradas, Página de 4096 bytes
----------------------------------------
Estatísticas da Simulação da TLB
----------------------------------------
Total de Acessos à Memória: 1,048,576
Acertos (Hits) na TLB:       0 (0.00%)
Faltas (Misses) na TLB:      1,048,576 (100.00%)
  - Misses Compulsórios:     1,024
  - Misses de Capacidade:    1,047,552
----------------------------------------
Razão (Não-Compulsórios / Total de Misses): 99.9